<a href="https://colab.research.google.com/github/KishenPatel97/Book_Summarization/blob/main/bleu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install bleu
from bleu import list_bleu
# from summarize import processBook

In [16]:
hypothesis = "the #### transcript is a written version of each day 's cnn student news program use this transcript to help students with reading comprehension and vocabulary use the weekly newsquiz to test your knowledge of storie s you saw on cnn student news"

reference = "this page includes the show transcript use the transcript to help students with reading comprehension and vocabulary at the bottom of the page, comment for a chance to be mentioned on cnn student news. you must be a teacher or a student age # # or older to request a mention on the cnn student news roll call . the weekly newsquiz tests students' knowledge of even ts in the news"

# If you want to check only one hypothesis (a list of sentences)
list_bleu([reference], [hypothesis])

17.77

In [ ]:
from bleu import file_bleu

control1 = processBook('texts/cn_alice.txt')
control2 = processBook('texts/gs_alice.txt')